<a href="https://colab.research.google.com/github/ekdnam/NLP-Intro/blob/master/iMDB/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

import sys
print(sys.version)

2.2.0
3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


In [0]:
# !pip install tensorflow_datasets

In [0]:
import tensorflow_datasets as tfds

imdb, data = tfds.load('imdb_reviews', with_info = True, as_supervised = True)

In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [6]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received '
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [9]:
num_epochs = 10
model.fit(padded, 
          training_labels_final, 
          epochs=num_epochs, 
          validation_data=(testing_padded, testing_labels_final)
          )

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4959 - accuracy: 0.7397 - val_loss: 0.3511 - val_accuracy: 0.8474
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2398 - accuracy: 0.9075 - val_loss: 0.3687 - val_accuracy: 0.8410
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0905 - accuracy: 0.9770 - val_loss: 0.4504 - val_accuracy: 0.8288
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0227 - accuracy: 0.9973 - val_loss: 0.5270 - val_accuracy: 0.8286
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0066 - accuracy: 0.9996 - val_loss: 0.5997 - val_accuracy: 0.8268
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.6340 - val_accuracy: 0.8305
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 9.0773e-04 - accuracy: 1.0000 - val_loss: 0.6786 - val_accuracy: 0.8316
Ep

In [10]:
model_1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_1.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [11]:
num_epochs = 10
model_1.fit(padded, 
          training_labels_final, 
          epochs=num_epochs, 
          validation_data=(testing_padded, testing_labels_final)
          )

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5505 - accuracy: 0.7594 - val_loss: 0.3881 - val_accuracy: 0.8388
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.3346 - accuracy: 0.8606 - val_loss: 0.3313 - val_accuracy: 0.8589
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2720 - accuracy: 0.8894 - val_loss: 0.3296 - val_accuracy: 0.8571
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2360 - accuracy: 0.9075 - val_loss: 0.3479 - val_accuracy: 0.8509
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2099 - accuracy: 0.9218 - val_loss: 0.3635 - val_accuracy: 0.8490
Epoch 6/10
782/782 [==============================] - 4s 4ms/step - loss: 0.1889 - accuracy: 0.9293 - val_loss: 0.3898 - val_accuracy: 0.8432
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 0.1710 - accuracy: 0.9382 - val_loss: 0.4197 - val_accuracy: 0.8379
Epoch 

In [17]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model_2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_2.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_4 ( (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 102       
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 14        
Total params: 160,116
Trainable params: 160,116
Non-trainable params: 0
_________________________________________________________________


In [18]:
num_epochs = 10
model_2.fit(padded, 
          training_labels_final, 
          epochs=num_epochs, 
          validation_data=(testing_padded, testing_labels_final)
          )

Epoch 1/10
782/782 [==============================] - 4s 4ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5065
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.6931 - accuracy: 0.4995 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.6931 - accuracy: 0.4957 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.6931 - accuracy: 0.4971 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.6931 - accuracy: 0.4965 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.6931 - accuracy: 0.4997 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 0.6931 - accuracy: 0.5030 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 

In [25]:
model_3 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(8, activation='relu'),
    #tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_3.summary()


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_9 ( (None, 16)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 9         
Total params: 160,145
Trainable params: 160,145
Non-trainable params: 0
_________________________________________________________________


In [26]:
num_epochs = 10
model_3.fit(padded, 
          training_labels_final, 
          epochs=num_epochs, 
          validation_data=(testing_padded, testing_labels_final)
          )

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5503 - accuracy: 0.7505 - val_loss: 0.3824 - val_accuracy: 0.8423
Epoch 2/10
782/782 [==============================] - 4s 4ms/step - loss: 0.3304 - accuracy: 0.8638 - val_loss: 0.3323 - val_accuracy: 0.8570
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2685 - accuracy: 0.8925 - val_loss: 0.3361 - val_accuracy: 0.8548
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2325 - accuracy: 0.9096 - val_loss: 0.3475 - val_accuracy: 0.8525
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2073 - accuracy: 0.9215 - val_loss: 0.3719 - val_accuracy: 0.8463
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1859 - accuracy: 0.9322 - val_loss: 0.3963 - val_accuracy: 0.8417
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1685 - accuracy: 0.9407 - val_loss: 0.4363 - val_accuracy: 0.8345
Epoch 